In [1]:
import pandas as pd
import glob

Unire tutti i dati nei CSV files dentro `data` in un singolo Pandas dataframe in memoria.

In [2]:
columns_to_keep = [
    "artist",
    "firstName",
    "gender",
    "itemInSession",
    "lastName",
    "length",
    "level",
    "location",
    "sessionId",
    "song",
    "userId",
]
dfs = []
for csv_f in glob.glob("data/*.csv"):
    dfs.append(pd.read_csv(csv_f, index_col=None, header=0))

df = pd.concat(dfs, axis=0, ignore_index=True)
df = df[columns_to_keep]
df.dropna(how="any", inplace=True)
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26.0
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26.0
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26.0
5,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61.0
9,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80.0


#### Creare un Cluster

Per eseguire Apache Cassandra all'interno di un contenitore Docker, procedere come segue:

1. Assicurarsi che Docker sia installato localmente.
2. Eseguire `docker pull cassandra` per recuperare l'ultima immagine da DockerHub.
3. Eseguire `docker run --name cassandra-container -p 9042:9042 -d cassandra:latest` per avviare un container sulla porta 9042.
4. Per fermare il contenitore, eseguire `docker stop cassandra-container`.

Se Apache Cassandra è installato altrove, l'indirizzo IP all'interno dell'oggetto Cluster deve essere modificato di conseguenza.


In [3]:
from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])
session = cluster.connect()

Creare una Keyspace


In [4]:
try:
    session.execute(
        """CREATE KEYSPACE IF NOT EXISTS demo0201db
                        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

Impostare un Keyspace


In [5]:
try:
    session.set_keyspace("demo0201db")
except Exception as e:
    print(e)

## Query 1:

`Forniscimi l'artista, il titolo della canzone e la lunghezza della canzone nella cronologia dell'applicazione musicale che è stata ascoltata durante sessionId = 338 e itemInSession = 4`.

Poiché la query filtra per `sessionId` e `itemInSession`, questi campi devono essere inclusi nella `PRIMARY KEY`.


In [6]:
# set up song_info_by_session and select statement
create_query1_tbl = """CREATE TABLE IF NOT EXISTS song_info_by_session
                       (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession));"""
try:
    session.execute(create_query1_tbl)
except Exception as e:
    print(e)

query1 = """SELECT artist, song, length FROM song_info_by_session 
            WHERE sessionId=338 AND itemInSession=4;"""

In [7]:
# populate song_info_by_session
for i,row in df.iterrows():
    query = "INSERT INTO song_info_by_session (sessionId, itemInSession, artist, song, length) VALUES "
    query = query + "(%s, %s, %s, %s, %s);"
    session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], row[5]))

In [8]:
## inspect song_info_by_session content
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    res = (row.artist, row.song, str(row.length))
    print("|".join(res))

Faithless|Music Matters (Mark Knight Dub)|495.30731201171875


## Query2:

`Dammi solo i seguenti dati: nome dell'artista, canzone (ordinati per itemInSession) e utente (nome e cognome) per userid = 10, sessionid = 182`.

`sessionId` e `userId` sono usati per il filtraggio, ma `itemInSession` è usato per l'ordinamento. I primi due file sono stati raggruppati insieme, mentre il terzo viene utilizzato come colonna di raggruppamento.


In [9]:
# set up song_info_by_user_session and select statement
create_query2_tbl = """CREATE TABLE IF NOT EXISTS song_info_by_user_session
                       (sessionId int, userId int, itemInSession int, artist text, song text, user text, PRIMARY KEY((sessionId, userId), itemInSession));"""
try:
    session.execute(create_query2_tbl)
except Exception as e:
    print(e)

query2 = """SELECT artist, song, user FROM song_info_by_user_session
            WHERE sessionId=182 AND userId=10;"""

## populate song_info_by_user_session
for i,row in df.iterrows():
    query = "INSERT INTO song_info_by_user_session (sessionId, userId, itemInSession, artist, song, user) VALUES "
    query = query + "(%s, %s, %s, %s, %s, %s);"
    session.execute(
        query,
        (
            int(row[8]),
            int(row[10]),
            int(row[3]),
            row[0],
            row[9],
            row[1] + " " + row[4],
        ),
    )

## inspect song_info_by_user_session content
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    res = (row.artist, row.song, row.user)
    print("|".join(res))

Down To The Bone|Keep On Keepin' On|Sylvie Cruz
Three Drives|Greece 2000|Sylvie Cruz
Sebastien Tellier|Kilometer|Sylvie Cruz
Lonnie Gordon|Catch You Baby (Steve Pitron & Max Sanna Radio Edit)|Sylvie Cruz


Pulire l'ambiente di sviluppo


In [10]:
try:
    session.execute("DROP TABLE song_info_by_session;")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE song_info_by_user_session;")
except Exception as e:
    print(e)

session.shutdown()
cluster.shutdown()
